In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Dense, Flatten
from keras.models import Sequential

In [ ]:
os.chdir("/kaggle/input")

training_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False
)

testing_data = ImageDataGenerator(rescale=1./255)

## Load training and testing data
## Specify classes of images and target size (28*28 pixel images)
train = training_data.flow_from_directory(
    'dataset/train/',
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=128,
    classes=['zero', 'one', 'two', 'three', 'four', 'five', 'ok', 'up', 'down'],
    class_mode='categorical'
)

test = testing_data.flow_from_directory(
    'dataset/test/',
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=128,
    classes=['zero', 'one', 'two', 'three', 'four', 'five', 'ok', 'up', 'down'],
    class_mode='categorical'
)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(9))
model.add(Activation('softmax'))

model.summary()


In [ ]:
## Using Adadelta optimizer and Categorical crossentropy for loss function
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
## Train model
model.fit_generator(
    train,
    steps_per_epoch=120,
    epochs=40,
    validation_data=test,
    validation_steps=28
)

In [ ]:
## Save Model
model.save('/kaggle/working/hand_detection_model_3.h5')